Extracting and comparing the generated rules 

In [19]:
def maybe_replace_variable(rule):
    all_variables = ["?a", "?b", "?g", "?h"]
    for var1 in all_variables[::-1]:
        if var1 in rule:
            for var2 in all_variables:
                if not var2 in rule:
                    rule = rule.replace(var1, var2)
                    break
                if var2 >= var1:
                    break
    return rule

In [20]:
def compare_rules_amie(rules_before, rules_after):
    with open(rules_before) as file:
        Rules_1 = file.read().strip().split('\n')
    with open(rules_after) as file:
        Rules_2 = file.read().strip().split('\n')
    rules_1 = list(filter(lambda x: x.startswith("?"), Rules_1))
    rules_2 = list(filter(lambda x: x.startswith("?"), Rules_2))
    rules_1 = list(map(maybe_replace_variable, rules_1))
    rules_2 = list(map(maybe_replace_variable, rules_2))
    Rules_1 = set(map(lambda x: x.split('\t')[0], rules_1))
    Rules_2 = set(map(lambda x: x.split('\t')[0], rules_2))
    
    new_rules = Rules_2-Rules_1
    dropped_rules = Rules_1-Rules_2
    
    same_rules = Rules_1.intersection(Rules_2)
    
    new_rules = sorted(filter(lambda x: x.split('\t')[0] in new_rules, rules_2),key=lambda t: float(t.split('\t')[3]), reverse=True)
    dropped_rules = sorted(filter(lambda x: x.split('\t')[0] in dropped_rules, rules_1),key=lambda t: float(t.split('\t')[3]), reverse=True)
    same_rules_1 = sorted(filter(lambda x: x.split('\t')[0] in same_rules, rules_1),key=lambda t: float(t.split('\t')[3]), reverse=True)
    same_rules_2 = sorted(filter(lambda x: x.split('\t')[0] in same_rules, rules_2),key=lambda t: float(t.split('\t')[3]), reverse=True)
    rules_1 = sorted(rules_1, key=lambda t: float(t.split('\t')[3]), reverse=True)
    rules_2 = sorted(rules_2, key=lambda t: float(t.split('\t')[3]), reverse=True)
    
    return same_rules_1, same_rules_2, rules_1, rules_2, new_rules, dropped_rules

Writing the statistics of the generated rules into a file (latex table).

In [21]:
def write_rule_statistics(approach="TransAMIE"):
    begin_table = "\\begin{table*}[h]\n\caption{Counts of rules generated on each knowledge graph}\n\label{tab:generated}\n\\resizebox{\\textwidth}{!}{\\begin{tabular}{|l|r|r|r|r|r|}\n \hline \n"
    table_head = "\multicolumn{6}{|c|}{Topk = 50} \\\\ \n\hline \n\multicolumn{1}{|c|}{Dataset} & \multicolumn{1}{c|}{\#Rules Before} & \multicolumn{1}{c|}{\#Rules After} & \multicolumn{1}{c|}{\#New Rules} & \multicolumn{1}{c|}{\#Dropped Rules} & \multicolumn{1}{c|}{\#Same Rules} \\\\ \n \hline \n"
    table = begin_table
    for topk in [50, 500, 5000]:
        if topk == 500:
            table_head = "\multicolumn{6}{|c|}{Topk = 500} \\\\ \n" + "\hline \n"
        if topk == 5000:
            table_head = "\multicolumn{6}{|c|}{Topk = 5000} \\\\ \n" + "\hline \n"
        table += table_head
        for kg in ['wn18rr', 'carcinogenesis', 'mutagenesis', 'fb15k', 'yago3', 'drkg', 'openbiolink']:
            all_rules = compare_rules_amie(f"data/AMIE/rules_{kg}/rules.txt", f"data/{approach}/rules_{kg}/rules_top{topk}.txt")
            if kg == "yago3":
                kg_name = kg.upper()+"-10"
            elif kg == "fb15k":
                kg_name = kg.upper()+"-237"
            else:
                kg_name = kg.upper()
            table += f"{kg_name}  & {len(all_rules[2])} & {len(all_rules[3])} & {len(all_rules[-2])} & {len(all_rules[-1])} & {len(all_rules[0])}\\\\ \n\hline \n"
    table += "\end{tabular}}\n\end{table*}"
    with open(f"data/{approach}/rules_statistics_table.txt", "w") as file:
        file.write(table)

In [22]:
write_rule_statistics()

In [23]:
write_rule_statistics("DistAMIE")

In [24]:
write_rule_statistics("RotAMIE")

### Confidence scores

Table template

\begin{table*}[h]
\caption{Average confidence scores}
\label{tab:confidence}
 \resizebox{\textwidth}{!}{\begin{tabular}{|l|cc|cc|cc|cc|cc|}
\hline
\multicolumn{11}{|c|}{Topk = 50} \\ 
\hline
\multirow{2}{*}{Dataset} & \multicolumn{2}{c|}{\textbf{Rules before}}           & \multicolumn{2}{c|}{\textbf{Rules after}} & \multicolumn{2}{c|}{\textbf{New rules}}    & \multicolumn{2}{c|}{\textbf{Dropped rules}}  & \multicolumn{2}{c|}{\textbf{Same rules}} \\ \cline{2-11} 
& \multicolumn{1}{p{1.5cm}}{\centering Std.\\ conf.}    & \multicolumn{1}{p{1.5cm}|} {\centering PCA \\ conf. }  & \multicolumn{1}{p{1.5cm}}{\centering Std. \\ conf.} & \multicolumn{1}{p{1.5cm}|}{\centering PCA \\ conf.}  & \multicolumn{1}{p{1.5cm}}{\centering Std.\\  conf.} & \multicolumn{1}{p{1.5cm}|} {\centering PCA \\  conf.}  & \multicolumn{1}{p{1.5cm}}{\centering Std. \\  conf.} & \multicolumn{1}{p{1.5cm}|}{\centering PCA \\ conf.} & \multicolumn{1}{p{1.5cm}}{\centering Std. \\  conf.} & \multicolumn{1}{p{1.5cm}|}{\centering PCA \\ conf.} \\ \hline
WN18RR  & 0.247  & 0.487     & \textbf{0.541} & \textbf{0.778} & \textbf{0.655} & \textbf{0.888} & 0.158      & 0.406     & 0.284 & 0.520  \\ 
\hline
CARCINOGENESIS  & \textbf{0.516} & \textbf{0.738} & 0.445 & 0.695  & 0.415 & 0.681   & 0.528  & 0.754  & 0.513  & 0.733     \\ \hline
MUTAGENESIS  & 0.437 & 0.543 & \textbf{0.473} & \textbf{0.553} & \textbf{0.473} & \textbf{0.563} & {–}  & – & 0.437  & 0.543 \\ \hline
FB15K-237   & \textbf{0.307} & \textbf{0.563} & 0.271  & 0.563  & 0.152  & 0.546  & – & –  & 0.307  & 0.563     \\ \hline
YAGO3-10 & 0.192 & 0.324 & \textbf{0.601} & \textbf{0.671} & \textbf{0.858} & \textbf{0.888} & 0.148 & 0.290  & 0.236  & 0.360     \\ \hline
DRKG & 0.170  & \textbf{0.289} & \textbf{0.172} & 0.287  & 0.147 & 0.271  & 0.143 & 0.273     & 0.201 & 0.308 \\ \hline
OPENBIOLINK & 0.280 & 0.317 & \textbf{0.291} & \textbf{0.335} & \textbf{0.580} & \textbf{0.650} & 0.216 & 0.248  & 0.306 & 0.344     \\ \hline
\end{tabular}}
\end{table*}

Reading and extracting the confidence scores of the generated rules 

In [25]:
def extract_confidence(rules: list):
    try:
        pca_conf = round(sum(list(map(lambda x: float(x.split('\t')[3]), rules)))/len(rules), 2)
        std_conf = round(sum(list(map(lambda x: float(x.split('\t')[2]), rules)))/len(rules), 2)
    except ZeroDivisionError:
        return "-", "-"
    return std_conf, pca_conf

In [26]:
def write_avg_confidence(approach="TransAMIE"):
    begin_table = "\\begin{table*}[h]\n\caption{Average confidence scores}\n\label{tab:confidence}\n\\resizebox{\\textwidth}{!}{\\begin{tabular}{|l|cc|cc|cc|cc|cc|}\n \hline \multicolumn{11}{|c|}{Topk = 50} \\\\ \hline \n"
    table_head = " \multirow{2}{*}{Dataset} & \multicolumn{2}{c|}{\\textbf{Rules before}} & \multicolumn{2}{c|}{\\textbf{Rules after}} & \multicolumn{2}{c|}{\\textbf{New rules}}    & \multicolumn{2}{c|}{\\textbf{Dropped rules}}" + \
    "& \multicolumn{2}{c|}{\\textbf{Same rules}} \\\\ \cline{2-11} & \multicolumn{1}{p{1.5cm}}{\centering Std.\\ conf.}    & \multicolumn{1}{p{1.5cm}|} {\centering PCA \\ conf. }  & \multicolumn{1}{p{1.5cm}}{\centering Std. \\ conf.}" + \
    "& \multicolumn{1}{p{1.5cm}|}{\centering PCA \\ conf.}  & \multicolumn{1}{p{1.5cm}}{\centering Std.\\  conf.} & \multicolumn{1}{p{1.5cm}|} {\centering PCA \\  conf.}  & \multicolumn{1}{p{1.5cm}}{\centering Std. \\  conf.} &" + \
    "\multicolumn{1}{p{1.5cm}|}{\centering PCA \\ conf.} & \multicolumn{1}{p{1.5cm}}{\centering Std. \\  conf.} & \multicolumn{1}{p{1.5cm}|}{\centering PCA \\ conf.} \\\\ \n\hline\n"
    table = begin_table
    for topk in [50, 500, 5000]:
        if topk == 500:
            table_head = "\multicolumn{11}{|c|}{Topk = 500} \\\\ \n"+ "\hline \n"
        if topk == 5000:
            table_head = "\multicolumn{11}{|c|}{Topk = 5000} \\\\ \n" + "\hline \n"
        table += table_head
        for kg in ['wn18rr', 'carcinogenesis', 'mutagenesis', 'fb15k', 'yago3', 'drkg', 'openbiolink']:
            same_rules, _, rules_before, rules_after, new_rules, dropped_rules = compare_rules_amie(f"data/AMIE/rules_{kg}/rules.txt", f"data/{approach}/rules_{kg}/rules_top{topk}.txt")
            if kg == "yago3":
                kg_name = kg.upper()+"-10"
            elif kg == "fb15k":
                kg_name = kg.upper()+"-237"
            else:
                kg_name = kg.upper()
            table += f"{kg_name} "
            for rule in [rules_before, rules_after, new_rules, dropped_rules, same_rules]:
                std_conf, pca_conf = extract_confidence(rule)
                if 2<len(str(std_conf))<4:
                    std_conf = str(std_conf)+"0"
                if 2<len(str(pca_conf))<4:
                    pca_conf = str(pca_conf)+"0"
                table += f"& {std_conf} & {pca_conf} "
            table += "\\\\ \n\hline \n"
    table += "\end{tabular}}\n\end{table*}"
    with open(f"data/{approach}/confidence_table.txt", "w") as file:
        file.write(table)

In [27]:
write_avg_confidence()

In [28]:
write_avg_confidence("DistAMIE")

In [29]:
write_avg_confidence("RotAMIE")

Extract and write (embedding-based) link prediction results

In [30]:
def extract_results_from_file():
    import json
    begin_table = "\\begin{table*}[h]\n\caption{Link prediction results on the test set}\n\label{tab:link-prediction}\n\\resizebox{\\textwidth}{!}{\\begin{tabular}{|l|r|r|r|r}\n \hline \n"
    table_head = "\multicolumn{5}{|c|}{TransE} \\\\ \n\hline \n\multicolumn{1}{|c|}{Dataset} & \multicolumn{1}{c|}{Hits@1} & \multicolumn{1}{c|}{Hits@3} & \multicolumn{1}{c|}{Hits@10} & \multicolumn{1}{c|}{MRR} \\\\ \n \hline \n"
    table = begin_table
    for model in ["TransE", "DistMult", "RotatE"]:
        if model != "TransE":
            table_head = "\multicolumn{5}{|c|}{model} \\\\ \n".replace("model", model)+ "\hline \n"
        table += table_head
        for kg in ['wn18rr', 'carcinogenesis', 'mutagenesis', 'fb15k', 'yago3', 'drkg', 'openbiolink']:
            if kg == "yago3":
                kg_name = kg.upper()+"-10"
            elif kg == "fb15k":
                kg_name = kg.upper()+"-237"
            else:
                kg_name = kg.upper()
            table += f"{kg_name} "
            with open(f"data/embeddings/{kg}_{model.lower()}/results.json") as file:
                _ = json.load(file)
                results = _["metrics"]
            hits1, hits3 = round(results["both"]["realistic"]["hits_at_1"], 2), round(results["both"]["realistic"]["hits_at_3"], 2)
            hits10, mrr = round(results["both"]["realistic"]["hits_at_10"], 2), round(results["both"]["realistic"]["inverse_harmonic_mean_rank"], 2)
            for metric in [hits1, hits3, hits10, mrr]:
                if len(str(metric))<4:
                    metric = str(metric)+"0"
                table += f"& {metric} "
            table += "\\\\ \n\hline \n"
    table += "\end{tabular}}\n\end{table*}"
    with open(f"data/link_prediction.txt", "w") as file:
        file.write(table)

In [31]:
extract_results_from_file()